We can use `python-bitcoinlib` to do a lot in the bitcoin space.

In [119]:
#!pip install python-bitcoinlib

In [128]:
import bitcoin
from bitcoin.rpc import Proxy
from bitcoin.core import b2lx, b2x

In [129]:
bitcoin.SelectParams('regtest')
bt = Proxy(btc_conf_file="/Users/sebastian.ortega/Sync/Main/btcmaster/devnode/testnet/bitcoin.conf")

We can access the same information as with `bitcoin-cli`.

In [130]:
bt.getinfo()

{'balance': 9999982660,
 'blocks': 102,
 'connections': 1,
 'difficulty': Decimal('4.656542373906925E-10'),
 'errors': '',
 'keypoololdest': 1509012464,
 'keypoolsize': 100,
 'paytxfee': 0,
 'protocolversion': 70015,
 'proxy': '',
 'relayfee': Decimal('0.00001000'),
 'testnet': False,
 'timeoffset': 0,
 'version': 130200,
 'walletversion': 130000}

In [11]:
bt.getmininginfo()

{'blocks': 101,
 'chain': 'regtest',
 'currentblocksize': 1000,
 'currentblocktx': 0,
 'currentblockweight': 4000,
 'difficulty': Decimal('4.656542373906925E-10'),
 'errors': '',
 'networkhashps': Decimal('9.513765660635558E-7'),
 'pooledtx': 0,
 'testnet': False}

This client parses the whole messages and is able to anything with the data. Bad news: more subtle details to watch for.

In [123]:
num_blocks = bt.getinfo()['blocks']
num_blocks

102

In [124]:
height = int(num_blocks / 3)
height

34

In [125]:
block_id = bt.getblockhash(height)
block_id

b'\x85 ,x\xb8\x80\xf4\x9b\x19\xaf>\x9c\xb6\xa0r\xbf\xb1)<u2w\xdaG\x161+H\xfdJ9m'

Note that the block is is **binary data**. In this case in little-endian format.

In [126]:
help(b2lx)
b2lx(block_id)

Help on function b2lx in module bitcoin.core:

b2lx(b)
    Convert bytes to a little-endian hex string
    
    Lets you show uint256's and uint160's the way the Satoshi codebase shows
    them.



'6d394afd482b311647da7732753c29b1bf72a0b69c3eaf199bf480b8782c2085'

In [28]:
block = bt.getblock(block_id)
block

CBlock(536870912, lx(11cb98c6a31c22b17228746097975b9690df17d4117a1e6e9cb1131d71acaf28), lx(c1ac66d88f6d8a68ae0685ca7225476468072ef21c8513126ae1417e7e96cc57), 1509012509, 0x207fffff, 0x00000001)

In [33]:
len(block.vtx)

1

In [35]:
tx = block.vtx[0]
tx

CTransaction((CTxIn(COutPoint(), CScript([x('21'), x('01')]), 0xffffffff),), (CTxOut(50.0*COIN, CScript([x('0348e1deaf69d7570a5ea7e762b8cbe8668f47b32b15dfda1420834744a15301b6'), OP_CHECKSIG])), CTxOut(0.0*COIN, CScript([OP_RETURN, x('aa21a9ede2f61c3f71d1defd3fa999dfa36953755c690689799962b48bebd836974e8cf9')]))), 0, 1)

In [40]:
b2lx(tx.GetTxid()) # Also in little endian

'c1ac66d88f6d8a68ae0685ca7225476468072ef21c8513126ae1417e7e96cc57'

In [47]:
output = tx.vout[0]

In [50]:
output.nValue

5000000000

In [63]:
list(output.scriptPubKey)

[b'\x03H\xe1\xde\xafi\xd7W\n^\xa7\xe7b\xb8\xcb\xe8f\x8fG\xb3+\x15\xdf\xda\x14 \x83GD\xa1S\x01\xb6',
 OP_CHECKSIG]

# Accessing the wallet

In [106]:
address = bt.getnewaddress()
address

P2PKHBitcoinAddress('myoTRSmiZZpNdvfoPqg3B9YgZd4YLGnzT9')

In [107]:
bt.getbalance()

5000000000

Let's send some funds to this new address.

In [116]:
tx = bt.sendtoaddress(addr=address, amount=100000, comment="meh")

In [117]:
b2lx(tx)

'bba9ebcc163d21acb4c465a70ef445ed50f705aea83e764dbf190a7ac2f0d786'

In [132]:
bt.gettransaction(tx)

{'amount': Decimal('0E-8'),
 'bip125-replaceable': 'no',
 'comment': 'meh',
 'confirmations': 0,
 'details': [{'abandoned': False,
   'account': '',
   'address': 'myoTRSmiZZpNdvfoPqg3B9YgZd4YLGnzT9',
   'amount': Decimal('-0.00100000'),
   'category': 'send',
   'fee': Decimal('-0.00004520'),
   'label': '',
   'vout': 1},
  {'account': '',
   'address': 'myoTRSmiZZpNdvfoPqg3B9YgZd4YLGnzT9',
   'amount': Decimal('0.00100000'),
   'category': 'receive',
   'label': '',
   'vout': 1}],
 'fee': Decimal('-0.00004520'),
 'hex': '0100000001e41dec26a5135d3d044ff5531b1ddb773608e7a6583a07f7e1d67dde069b992a010000006b483045022100f17a957d784ef7fafc75bbc5535bd5224f8b6ec27065164ee90507d2c188a4be0220223c7385b5b7843c3d9b176bab9c2a4fdcb73d6b6e11a2d3374756757bb1b6c30121028073e16f74115c4543034f9ccab8910eede36b92af0609996fbba7cabc23101efeffffff02c493ff29010000001976a914c09c520f8174324177cdaffb7aef543bdd076e2588aca0860100000000001976a914c8913bddd04cbde41bd38eb9f51b1955c8e0832488ac66000000',
 'time': 15090

In [131]:
help(bt.sendrawtransaction)

Help on method sendrawtransaction in module bitcoin.rpc:

sendrawtransaction(tx, allowhighfees=False) method of bitcoin.rpc.Proxy instance
    Submit transaction to local node and network.
    
    allowhighfees - Allow even if fees are unreasonably high.



# Creating transactions

In [133]:
help(bt.fundrawtransaction)

Help on method fundrawtransaction in module bitcoin.rpc:

fundrawtransaction(tx, include_watching=False) method of bitcoin.rpc.Proxy instance
    Add inputs to a transaction until it has enough in value to meet its out value.
    
    include_watching - Also select inputs which are watch only
    
    Returns dict:
    
    {'tx':        Resulting tx,
     'fee':       Fee the resulting transaction pays,
     'changepos': Position of added change output, or -1,
    }



In [ ]:
# sendrawtransaction, signrawtransaction